<a href="https://colab.research.google.com/github/rodiwaa/learnings-pocs/blob/main/notebooks/resume_rag_system_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install langchain langchain.openai openai langchain-community langsmith chromadb python-dotenv sentence-transformers

## API keys from .env

In [5]:
import os
from google.colab import drive
from dotenv import load_dotenv

drive.mount("/content/drive")

load_dotenv(dotenv_path="/content/drive/MyDrive/Projects/.env/.env")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


True

## Read pdf from drive
HOLD. will work with docs now, import PDF later.

## Create docs for "about me"

In [17]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.documents import Document
from langchain_community.vectorstores import Chroma
# from langchain_openai import OpenAIEmbeddings
# from sentence_transformers import SentenceTransformer # does not work well w langchain/chroma; use SentenceTransformerEmbeddings instead
from langchain_community.embeddings import SentenceTransformerEmbeddings

docs = [
    Document(page_content="Jack likes to build workflows and AI systems"),
    Document(page_content="Jack has worked on following technology stacks - Langchain, Langgraph, Langsmith"),
    Document(page_content="Jack is friends with Tom and Sally."),
    Document(page_content="Jack loves to watch music and listen to music.")
]

embedding_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

vectorstore = Chroma.from_documents(
    embedding = embedding_model,
    documents = docs,
    persist_directory = "resume-rag-docs-pd"
)

# check if docs are added
added_docs = vectorstore.get()

print(f"added {len(added_docs)} docs")
print(added_docs)

added 7 docs
{'ids': ['059e7efe-338b-458e-83c3-f37244280b02', 'd4cded3d-ab4c-49d4-8ff8-2ca88371072b', '821aee3b-7d12-4290-831c-bae9e949501b', '578e8951-80a4-4077-add2-d0d7651e426d', '6ab75bf1-9942-4941-840c-184867baba78', '9d6f5d9b-7eaf-4180-b1ed-900c77de44a7', '24bf093e-d2c5-46d3-82d9-1a29ad4edbd5', '58fbcf6c-abb9-47f1-a619-81eee2df47e0'], 'embeddings': None, 'documents': ['Jack likes to build workflows and AI systems', 'Jack has worked on following technology stacks - Langchain, Langgraph, Langsmith', 'Jack is friends with Tom and Sally.', 'Jack loves to watch music and listen to music.', 'Jack likes to build workflows and AI systems', 'Jack has worked on following technology stacks - Langchain, Langgraph, Langsmith', 'Jack is friends with Tom and Sally.', 'Jack loves to watch music and listen to music.'], 'uris': None, 'included': ['metadatas', 'documents'], 'data': None, 'metadatas': [None, None, None, None, None, None, None, None]}


### Retrieve docs from VS based on query sim searches

In [ ]:
# vector store sim search

# try sim search/ default, compress context search, MMR, compare results

# compare perf, text splittint types for better perf?

Quetions
- choosing dimensions for embedding
- chunking size